In [1]:
import serial
import re
import time
import pygame
import sys
import math

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Position:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0

class Orientation:
    def __init__(self):
        self.roll = 0
        self.pitch = 0

class Pose:
    def __init__(self):
        self.position = Position()
        self.orientation = Orientation()

    def update_pose(self, pose_array):
        self.position.x = pose_array[0]
        self.position.y = pose_array[1]
        self.position.z = pose_array[2]
        self.orientation.pitch = pose_array[3]
        self.orientation.roll = pose_array[4]

    def __str__(self):
        return f"X: {self.position.x}, Y: {self.position.y}, Z: {self.position.z}, P: {self.orientation.pitch}, R: {self.orientation.roll}"

In [7]:
import serial
import re
import time

from Pose import Pose

class Robot:
    def __init__(self, port):
        self.ser = serial.Serial(port)
        self.ser.write(b"CON \r")
        time.sleep(5)
        # self.ser.write(b"HOME \r")
        # time.sleep(160)
        response = self.ser.read_all()
        print(response.decode('ascii'))

        self.pose = Pose()  

        self.get_intial_position()

    def close(self):
        self.ser.close()

    def get_intial_position(self):
        self.ser.write(b"HERE P1 \r")
        time.sleep(1)
        self.ser.write(b"LISTPV P1 \r")
        time.sleep(1)
        response = self.ser.read_all()
        print(response)
        pattern = "X:([\s-]\d+)\s*Y:([\s-]\d+)\s*Z:([\s-]\d+)\s*P:([\s-]\d+)\s*R:([\s-]\d+)"
        # X: 3423, Y: -353, Z: 3423, P: -201, R: 76
        match = re.search(pattern, response.decode('ascii'))
        self.pose.update_pose([
            int(match.group(1)),
            int(match.group(2)),
            int(match.group(3)),
            int(match.group(4)),
            int(match.group(5))
        ])

    
    '''
    Starts from the initial position and updates the position
    '''

    def move_robot_x(self, increment, delay, delay2):

            self.pose.update_pose([
                self.pose.position.x + increment,
                self.pose.position.y,
                self.pose.position.z,
                self.pose.orientation.pitch,
                self.pose.orientation.roll,
            ])
            print(self.ser.read_all())
            position_command = f"SETPVC P1 X {self.pose.position.x} \r"
            self.ser.write(bytes(position_command, encoding='utf-8'))
            time.sleep(delay)
            print(self.ser.read_all())
            self.ser.write(b"MOVEL P1 \r")
            time.sleep(delay2)
            print(self.ser.read_all())

    def move_robot_y(self, increment, delay, delay2):

        self.pose.update_pose([
            self.pose.position.x,
            self.pose.position.y + increment,
            self.pose.position.z,
            self.pose.orientation.pitch,
            self.pose.orientation.roll,
        ])


        print(self.ser.read_all())
        position_command = f"SETPVC P1 Y {self.pose.position.y} \r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        time.sleep(delay)
        print(self.ser.read_all())
        self.ser.write(b"MOVEL P1 \r")
        time.sleep(delay2)
        print(self.ser.read_all())

    def move_robot_z(self, increment, delay, delay2):
        print("move z")
        self.pose.update_pose([
            self.pose.position.x,
            self.pose.position.y,
            self.pose.position.z + increment,
            self.pose.orientation.pitch,
            self.pose.orientation.roll,
        ])
        
        print(self.ser.read_all())
        position_command = f"SETPVC P1 Z {self.pose.position.z} \r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        time.sleep(delay)
        print(self.ser.read_all())
        self.ser.write(b"MOVEL P1 \r")
        time.sleep(delay2)
        print(self.ser.read_all())

    def move_robot_pitch(self, increment, delay, delay2):

        self.pose.update_pose([
            self.pose.position.x,
            self.pose.position.y,
            self.pose.position.z,
            self.pose.orientation.pitch + increment,
            self.pose.orientation.roll,
        ])

        print(self.ser.read_all())
        position_command = f"SETPVC P1 P {self.pose.orientation.pitch} \r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        time.sleep(delay)
        print(self.ser.read_all())
        self.ser.write(b"MOVEL P1 \r")
        time.sleep(delay2)
        print(self.ser.read_all())

    def move_robot_roll(self, increment, delay, delay2):

        self.pose.update_pose([
            self.pose.position.x,
            self.pose.position.y,
            self.pose.position.z,
            self.pose.orientation.pitch,
            self.pose.orientation.roll + increment,
        ])

        print(self.ser.read_all())
        position_command = f"SETPVC P1 R {self.pose.orientation.roll} \r"
        self.ser.write(bytes(position_command, encoding='utf-8'))
        time.sleep(delay)
        print(self.ser.read_all())
        self.ser.write(b"MOVEL P1 \r")
        time.sleep(delay2)
        print(self.ser.read_all())

    def print_pose(self):
        print(self.pose)

In [4]:
robot = Robot("COM5")

CON 
CONTROL ENABLED.
>
b'HERE P1 \r\n\x00Done.\r\n>LISTPV P1 \r\nPosition P1\r\n 1: 1       2: 394     3: 1961    4:-1959    5: 20     \r\n X: 7531    Y:-352     Z: 6726    P: 78      R:-199    \r\n>'


In [5]:
robot.move_robot_z(100, 0.3, 0.0)

move z
b'SETPVC P1 Z 6826 \r\n\x00Done.\r\n>'
b''


In [9]:
pygame.init()
pygame.joystick.init()

joystick = pygame.joystick.Joystick(0)
joystick.init()

running = True

r1_pressed = False
l1_pressed = False

while running:
    if (r1_pressed):
        robot.move_robot_z(50, 0.5, 0)
    
    if (l1_pressed):
        robot.move_robot_z(-50, 0.5, 0)
        
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            robot.close()
            pygame.quit()

        if event.type == pygame.JOYBUTTONDOWN:
            print(event)
            if event.button == 4:
                l1_pressed = True
            if event.button == 5:
                r1_pressed = True
        if event.type == pygame.JOYBUTTONUP:
            print(event)
            if event.button == 4:
                l1_pressed = False
            if event.button == 5:
                r1_pressed = False
    # time.sleep(1)

<Event(1539-JoyButtonDown {'joy': 0, 'instance_id': 0, 'button': 4})>
move z
b'MOVE P1 \r\n\x00Done.\r\n>SETPVC P1 Z 5826 '
b''
<Event(1540-JoyButtonUp {'joy': 0, 'instance_id': 0, 'button': 4})>
<Event(1539-JoyButtonDown {'joy': 0, 'instance_id': 0, 'button': 4})>
move z
b'MOVE P1 \r\n\x00Done.\r\n>SETPVC P1 Z 5776 '
b''
move z
b'MOVE P1 SETPVC P1 Z 5726 \r\n\r*** ERROR *** Undefined or illegal variable(s).\r\n*** Invalid command.\r\n>'
b''
<Event(1540-JoyButtonUp {'joy': 0, 'instance_id': 0, 'button': 4})>
<Event(1539-JoyButtonDown {'joy': 0, 'instance_id': 0, 'button': 4})>
move z
b'MOVE P1 \r\n\x00Done.\r\n>SETPVC P1 Z 5676 \r\n\x00Done.\r\n>'
b''
<Event(1540-JoyButtonUp {'joy': 0, 'instance_id': 0, 'button': 4})>
<Event(1539-JoyButtonDown {'joy': 0, 'instance_id': 0, 'button': 5})>
move z
b'MOVE P1 \r\n\x00Done.\r\n>SETPVC P1 Z 5726 \r\n\x00Done.\r\n>'
b''
move z
b'MOVE P1 SETPVC P1 Z 5776 \r\n\r*** ERROR *** Syntax error.\r\n*** Invalid command.\r\n>'
b''
<Event(1540-JoyButtonUp 

KeyboardInterrupt: 

In [ ]:
robot.close()